# 使用Pytorch进行目标检测

由于视频数据量过大，建议在co-lab上运行本节代码

这一节，我们将会重新训练一个已经运用在行人检测上的Mask-RCNN网络

它包含170个图像，其中包含345个行人实例，我们将用它来说明如何在torchvision中使用新功能，以便在自定义数据集上训练实例细分模型。

## 定义数据库

我们可以参考用于训练对象检测，实例细分和人员关键点检测的脚本，轻松添加新的自定义数据集。

数据集应该继承torch.utils.data.Dataset的标准形势，并应用 __len__ 和 __getitem__

In [1]:
import os
import numpy as np
import torch
from PIL import Image


class PennFudanDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages")))) # 提取imgs文件路径
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks")))) # 提取masks文件路径

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        # 提取mask数组中的不重复的数字，并排序
        obj_ids = obj_ids[1:]
        # 第一位是0，第二位是1，第三位..n，所以用第二位开始提取id

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]
        # 将mask数组中的数字转化为 True/False

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        # 计算有几个对象
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i]) # np.where 是寻找符合条件的位置，之前已经把mask数组中的数字转化为 True/False
            xmin = np.min(pos[1]) # pos[1] 存放的是x的位置
            xmax = np.max(pos[1])
            ymin = np.min(pos[0]) # pos[0] 存放的是y的位置
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # 计算对象框的面积
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

现在我们已经处理好自定义的数据了，接下来让我们来建立mask-RCNN模型吧

In [1]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /Users/fuqinwei/.cache/torch/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
